In [1]:
# Import the modified CSV file for better visualization.
# Note that the added sufixes (".1", ".2", ..., ".25") is added by pandas when read by doing df = pd.read_csv(csv_file).
# The actual CSV does not contain those sufixes.

import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sys

DATA_DIR = Path.home() / "projects" / "data"

csv_file = DATA_DIR / "processed" / "GSE290585_SeSaMeBeta_MM285_BS_simplified_columns.csv" # path to mouse csv file

print(sys.executable)
print(csv_file.exists())

df = pd.read_csv(csv_file)

with pd.option_context("display.max_columns", None,
                       "display.width", None):
        display(df.head(20))



/home/jullid/projects/mouse-DNA-project/.venv/bin/python
True


,probe_ID,Adrenal,Adrenal.1,Adrenal.2,Adrenal.3,Adrenal.4,Adrenal.5,Bile_Duct,Bile_Duct.1,Bile_Duct.2,Bile_Duct.3,Bile_Duct.4,Bile_Duct.5,Blood,Blood.1,Blood.2,Blood.3,Blood.4,Blood.5,Blood.6,Blood.7,Blood.8,Blood.9,Blood.10,Blood.11,Brain_Cortex,Brain_Cortex.1,Brain_Cortex.2,Brain_Cortex.3,Brain_Cortex.4,Brain_Cortex.5,Brain_Cortex.6,Brain_Cortex.7,Brain_Cortex.8,Brain_Cortex.9,Brain_Cortex.10,Brain_Cortex.11,Brain_Cortex.12,Brain_Cortex.13,Brain_Cortex.14,Brain_Cortex.15,Cerebellum,Cerebellum.1,Cerebellum.2,Cerebellum.3,Cerebellum.4,Cerebellum.5,Cerebellum.6,Cerebellum.7,Cerebellum.8,Colon,Colon.1,Colon.2,Colon.3,Colon.4,Colon.5,Colon.6,Colon.7,Colon.8,Colon.9,Colon.10,Diaphragm,Diaphragm.1,Diaphragm.2,Diaphragm.3,Diaphragm.4,Diaphragm.5,Ears,Ears.1,Ears.2,Ears.3,Ears.4,Ears.5,Ears.6,Ears.7,Eye,Eye.1,Eye.2,Eye.3,Eye.4,Eye.5,Eye.6,Eye.7,Eye.8,Eye.9,Femur,Femur.1,Femur.2,Femur.3,Femur.4,Femur.5,Femur.6,Femur.7,Femur.8,Femur.9,Heart,Heart.1,Heart.2,Heart.3,Heart.4,Heart.5,Heart.6,Heart.7,Heart.8,Heart.9,Kidney,Kidney.1,Kidney.2,Kidney.3,Kidney.4,Kidney.5,Kidney.6,Kidney.7,Kidney.8,Kidney.9,Kidney.10,Kidney.11,Liver,Liver.1,Liver.2,Liver.3,Liver.4,Liver.5,Liver.6,Liver.7,Liver.8,Liver.9,Liver.10,Liver.11,Liver.12,Liver.13,Liver.14,Liver.15,Liver.16,Liver.17,Liver.18,Liver.19,Liver.20,Liver.21,Liver.22,Liver.23,Liver.24,Liver.25,Lung,Lung.1,Lung.2,Lung.3,Lung.4,Lung.5,Lung.6,Lung.7,Lung.8,Lung.9,Mammary_Glands,Mammary_Glands.1,Mammary_Glands.2,Mammary_Glands.3,Mammary_Glands.4,Mammary_Glands.5,Optic_Nerve,Optic_Nerve.1,Optic_Nerve.2,Optic_Nerve.3,Optic_Nerve.4,Optic_Nerve.5,Optic_Nerve.6,Optic_Nerve.7,Optic_Nerve.8,Pancreas,Pancreas.1,Pancreas.2,Pancreas.3,Pancreas.4,Pancreas.5,Pancreas.6,Retina,Retina.1,Retina.2,Retina.3,Retina.4,Retina.5,Retina.6,Sciatic_Nerve,Sciatic_Nerve.1,Sciatic_Nerve.2,Sciatic_Nerve.3,Sciatic_Nerve.4,Skin,Skin.1,Skin.2,Skin.3,Skin.4,Skin.5,Skin.6,Skin.7,Skin.8,Skin.9,Skin.10,Spinal_Cord,Spinal_Cord.1,Spinal_Cord.2,Spinal_Cord.3,Spinal_Cord.4,Spinal_Cord.5,Spleen,Spleen.1,Spleen.2,Spleen.3,Spleen.4,Spleen.5,Spleen.6,Spleen.7,Spleen.8,Spleen.9,Spleen.10,Stomach,Stomach.1,Stomach.2,Stomach.3,Stomach.4,Subcortical_Brain,Subcortical_Brain.1,Subcortical_Brain.2,Subcortical_Brain.3,Subcortical_Brain.4,Subcortical_Brain.5,Subcortical_Brain.6,Subcortical_Brain.7,Subcortical_Brain.8,Tail,Tail.1,Tail.2,Tail.3,Tail.4,Tail.5,Tail.6,Tail.7,Tail.8,Tail.9,Tail.10,Tail.11,Testis,Testis.1,Testis.2,Testis.3,Testis.4,Testis.5,Testis.6,Testis.7,Testis.8,Testis.9,Thymus,Thymus.1,Thymus.2,Thymus.3,Thymus.4,Thymus.5,Thymus.6,Urinary_Bladder,Urinary_Bladder.1,Urinary_Bladder.2,Urinary_Bladder.3,Urinary_Bladder.4,Urinary_Bladder.5,Uterus,Uterus.1,Uterus.2,Uterus.3
0,cg00101675_BC21,0.694,0.782,0.593,0.525,0.609,0.626,0.631,0.600,0.600,0.602,0.581,0.573,0.783,0.793,0.827,0.808,0.784,0.861,0.792,0.802,0.815,0.860,0.834,0.798,0.736,0.758,0.742,0.783,0.718,0.814,0.751,0.753,0.847,0.832,0.847,0.822,0.868,0.833,0.851,0.756,0.837,0.830,0.890,0.899,0.858,0.870,0.887,0.888,0.778,0.725,0.750,0.742,0.785,0.659,0.743,0.797,0.680,0.545,0.554,0.569,0.707,0.690,0.660,0.701,0.658,0.691,0.705,0.637,0.733,0.742,0.530,0.561,0.582,0.513,0.813,0.766,0.877,0.841,0.854,0.897,0.827,0.841,0.803,0.814,0.510,0.535,0.635,0.694,0.740,0.696,0.588,0.636,0.568,0.675,0.597,0.621,0.736,0.741,0.740,0.700,0.705,0.684,0.750,0.727,0.650,0.703,0.654,0.724,0.754,0.774,0.717,0.694,0.598,0.528,0.570,0.514,0.705,0.720,0.735,0.593,0.638,0.709,0.714,0.757,0.677,0.678,0.754,0.665,0.666,0.729,0.680,0.718,0.756,0.758,0.722,0.708,0.729,0.758,0.767,0.752,0.688,0.725,0.725,0.657,0.775,0.771,0.736,0.726,0.733,0.842,0.731,0.732,0.837,0.758,0.727,0.696,0.692,0.724,0.775,0.782,0.802,0.799,0.823,0.831,0.839,0.787,0.807,0.682,0.781,0.770,0.654,0.722,0.619,0.681,0.773,0.747,0.764,0.744,0.711,0.771,0.732,0.723,0.749,0.557,0.657,0.527,0.739,0.696,0.643,0.629,0.699,0.691,0.663,0.639,0.742,0.795,0.713,0.666,0.718,0.620,0.650,0.570,0.633,0.815,0.813,0.808,0.790,0.759,0.800,0.866,0.687,0.674,0

In [19]:
# Collapse cell type columns into one averaged column per cell type, resulting in 29 averaged cell types/columns

probe_ids = df["probe_ID"]
df_drop_ID = df.drop(columns="probe_ID")

df_drop_ID.shape

# remove suffix .1 .2 etc. added by pandas
cell_type_labels = df_drop_ID.columns.str.split(".", n=1).str[0]

# collapse replicates
df_celltype = df_drop_ID.groupby(cell_type_labels, axis=1).mean()

df_celltype.shape


# Reattach probe_ID
df_celltype.insert(0, "probe_ID", probe_ids)

with pd.option_context("display.max_columns", None,
                       "display.width", None):
        display(df_celltype.head(20))

df_celltype.shape


/tmp/ipykernel_4127972/4258896439.py:12: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_celltype = df_drop_ID.groupby(cell_type_labels, axis=1).mean()


,probe_ID,Adrenal,Bile_Duct,Blood,Brain_Cortex,Cerebellum,Colon,Diaphragm,Ears,Eye,Femur,Heart,Kidney,Liver,Lung,Mammary_Glands,Optic_Nerve,Pancreas,Retina,Sciatic_Nerve,Skin,Spinal_Cord,Spleen,Stomach,Subcortical_Brain,Tail,Testis,Thymus,Urinary_Bladder,Uterus
0,cg00101675_BC21,0.638167,0.597833,0.813083,0.794438,0.859667,0.686273,0.684500,0.625375,0.8333,0.6277,0.7001,0.656667,0.711577,0.7428,0.739000,0.805000,0.701286,0.748857,0.6426,0.695364,0.642833,0.765182,0.6160,0.773667,0.619917,0.7340,0.845000,0.600167,0.36700
1,cg00116289_BC21,0.675667,0.656000,0.788250,0.708063,0.679778,0.675455,0.684500,0.546375,0.7374,0.7029,0.7649,0.685167,0.776077,0.7806,0.707833,0.734889,0.721286,0.664714,0.6144,0.678909,0.630000,0.761545,0.7298,0.721111,0.644917,0.7772,0.832429,0.716667,0.69225
2,cg00211372_TC21,0.680500,0.689667,0.748083,0.750812,0.753444,0.638182,0.687667,0.640250,0.7652,0.7370,0.7337,0.685250,0.751577,0.7590,0.698667,0.730444,0.675714,0.685714,0.6150,0.722545,0.671333,0.716364,0.6602,0.723111,0.710417,0.7471,0.807714,0.700167,0.70975
3,cg00531009_BC21,0.774667,0.766833,0.805000,0.694625,0.742444,0.726727,0.746000,0.734000,0.8349,0.7690,0.8165,0.747000,0.773192,0.8016,0.786667,0.805111,0.767714,0.779143,0.7420,0.740000,0.757667,0.760091,0.7666,0.750222,0.733000,0.7349,0.795000,0.797500,0.77300
4,cg00747726_TC21,0.721000,0.749000,0.802583,0.804250,0.784556,0.617182,0.726333,0.689000,0.6762,0.7327,0.7725,0.764000,0.821000,0.7779,0.763167,0.847667,0.770714,0.571857,0.7148,0.692364,0.768500,0.753636,0.7254,0.805889,0.719833,0.8073,0.824143,0.702667,0.69825
5,cg00896209_TC21,0.778333,0.756667,0.845167,0.819500,0.804778,0.721545,0.761833,0.754000,0.8415,0.7964,0.8001,0.765917,0.786654,0.8093,0.800000,0.828889,0.763000,0.805429,0.7476,0.748273,0.781167,0.793727,0.7776,0.812000,0.755750,0.8345,0.844857,0.815833,0.78325
6,cg00910419_BC21,0.048333,0.049333,0.046000,0.047375,0.041333,0.038727,0.050667,0.046125,0.0428,0.0555,0.0509,0.044583,0.053115,0.0541,0.059333,0.041111,0.052571,0.052143,0.0462,0.044818,0.049000,0.043273,0.0580,0.047667,0.040250,0.0457,0.051714,0.056667,0.06300
7,cg00920372_BC21,0.470167,0.276000,0.439667,0.400500,0.202333,0.211273,0.334667,0.201500,0.3536,0.4066,0.3756,0.357667,0.573500,0.3729,0.370500,0.291556,0.284000,0.312143,0.2404,0.177000,0.252667,0.391455,0.3376,0.362000,0.142583,0.2029,0.463571,0.288000,0.24025
8,cg01326836_BC21,0.071833,0.088000,0.134500,0.088750,0.061556,0.063545,0.089667,0.081500,0.0771,0.1446,0.0938,0.075000,0.109962,0.1125,0.108000,0.081222,0.088429,0.062286,0.0836,0.105364,0.076500,0.108818,0.1096,0.087444,0.050917,0.0532,0.143571,0.053000,0.08075
9,cg01340195_BC21,0.666000,0.613500,0.807583,0.560000,0.347667,0.582364,0.561500,0.579000,0.2815,0.6983,0.7397,0.479167,0.747846,0.6731,0.740667,0.621222,0.674571,0.130000,0.5832,0.531727,0.454500,0.749273,0.6016,0.537222,0.429417,0.7628,0.806000,0.377000,0.43800


(296070, 30)

In [ ]:
# 
#hm not sure if this is correct
target = df_celltype["Liver"]
background = df_celltype.drop(columns=["probe_ID", "Liver"]).mean(axis=1)
diff = background - target

diff # why is ts negative

diff_sorted = diff.sort_values(ascending=False)
diff_sorted

viz_df = pd.DataFrame({
    "target_meth": target,
    "background_meth": background,
    "diff": diff
})



viz_df_sorted = viz_df.sort_values(["diff"], ascending=False)
print(viz_df_sorted.shape)
viz_df_sorted.head(30)

(296070, 3)


,target_meth,background_meth,diff
173815,0.163038,0.785939,0.622900
67419,0.199077,0.800684,0.601607
99147,0.238538,0.834709,0.596170
126545,0.245385,0.833048,0.587664
200449,0.338231,0.921545,0.583314
228194,0.201115,0.775590,0.574474
198404,0.333500,0.895045,0.561545
21548,0.330615,0.886546,0.555931
167153,0.292115,0.847658,0.555542
100062,0.326423,0.880462,0.554039


In [20]:
"""
Script to loop through each
"""
# -------------------------------
# Configuration
# -------------------------------

TARGET_THRESHOLD = 0.3
BACKGROUND_THRESHOLD = 0.7

# -------------------------------
# Setup
# -------------------------------

probe_ids = df_celltype["probe_ID"]
tissues = df_celltype.columns.drop("probe_ID")

all_tissues_results = []  # we will stack results here

# -------------------------------
# Loop over tissues
# -------------------------------

for tissue in tissues:

    # 1. Target methylation (per row / region)
    target = df_celltype[tissue]

    # 2. Background methylation (mean of all OTHER tissues, per row)
    background = (
        df_celltype
        .drop(columns=["probe_ID", tissue])
        .mean(axis=1)
    )

    # 3. Differential score
    diff = background - target

    # 4. Assemble per-tissue DataFrame
    tissue_df = pd.DataFrame({
        "probe_ID": probe_ids,
        "tissue": tissue,
        "target_meth": target,
        "background_meth": background,
        "diff": diff
    })

    # --------------------------------------------------
    # SORTING STRATEGY A — PURE RANKING (NO FILTERING)
    # --------------------------------------------------
    tissue_df_sorted = tissue_df.sort_values(
        by="diff",
        ascending=False
    )

    # --------------------------------------------------
    # SORTING STRATEGY B — BIOLOGICAL FILTER + RANKING
    # (COMMENTED OUT — UNCOMMENT TO USE)
    # --------------------------------------------------
    # tissue_df_sorted = (
    #     tissue_df[
    #         (tissue_df["target_meth"] < TARGET_THRESHOLD) &
    #         (tissue_df["background_meth"] > BACKGROUND_THRESHOLD)
    #     ]
    #     .sort_values(by="diff", ascending=False)
    # )

    # 5. Store results
    all_tissues_results.append(tissue_df_sorted)

# -------------------------------
# Stack all tissues together
# -------------------------------

final_df = pd.concat(all_tissues_results, ignore_index=True)

# -------------------------------
# Final sanity checks
# -------------------------------

print("Final shape:", final_df.shape)
print(final_df.head())


Final shape: (8586030, 5)
          probe_ID   tissue  target_meth  background_meth      diff
0  cg29874695_BC21  Adrenal     0.217333         0.753007  0.535673
1  cg28499979_TC11  Adrenal     0.286000         0.820041  0.534041
2  cg45498816_BC11  Adrenal     0.250833         0.770463  0.519629
3  cg36101350_BC11  Adrenal     0.292000         0.801785  0.509785
4  cg34061439_BC21  Adrenal     0.160167         0.662750  0.502584


In [ ]:
# Next steps:
# Choose top 1000 regions, preferably through SORTING STRATEGY B (if it works). Choose carefully!! 
# VALIDATE THIS IS CORRECT!!!
# Maybe through t-test? if statistical difference.
